In [ ]:
import glob
import pandas as pd
import numpy as np 
from astropy.time import Time
from astropy.table import Table
from astropy.io import fits
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import copy
import time
from scipy.stats import sem
from datetime import datetime, timezone
from python_modules.muonpipe import usefull_func

In [ ]:
#muon_files = glob.glob('/Users/vdk/muons2024/all_muon_fits/*')
muon_files = glob.glob('/Users/vdk/muons2024/all_muon_fits/low_nsb_filter/*')
#print(sorted(muon_files))
#df_files = [pd.read_csv(muon_file, na_values=['NA', '?']) for muon_file in sorted(muon_files)[0:5]] # [0:5] for 19-22 years, [5:] for 22-23 years
df_files = [pd.read_csv(muon_file, na_values=['NA', '?']) for muon_file in muon_files[:4]] 
df_all_data = pd.concat(df_files, ignore_index=True)
#df_good_data = df_all_data[(df_all_data['muon_efficiency'] < 1) & (df_all_data['ring_containment'] > 0.999) & (df_all_data['size_outside'] < 500)]# & (df_all_data['ring_completeness'] > 0.8)]
df_good_data = df_all_data[(df_all_data['ring_containment'] > 0.999) & (df_all_data['size_outside'] < 500)]# & (df_all_data['ring_completeness'] > 0.8)]
df_good_data = df_all_data
del(df_all_data)
df_good_data

In [ ]:
# 14936835 without size_outside < 500
# 14935087 with  size_outside < 500
# 19138826 muons for all files
# 12508468 for 22-23 years but after cuts = 5647558
# 6630358 for 19-22 years
# 12 277 916 for aftercuts all years
# Only 59161 events if make cut on ring completeness > 0.999
# 2063237 events if make cut on ring completeness > 0.8
# 5336094 events for 19-22 years low nsb
# 2849682 events for 19-22 years low nsb ring_comp > 0.6
# 896399 events for 19-22 years low nsb ring_comp > 0.8
# 14933130 events for all years low nsb ring_cont>0.999
print(f"Start date {datetime.fromtimestamp(min(df_good_data['event_time'])).strftime('%Y-%m-%d %H:%M:%S.%f')}")
print(f"End date {datetime.fromtimestamp(max(df_good_data['event_time'])).strftime('%Y-%m-%d %H:%M:%S.%f')}")
#min(df_good_data['ring_completeness'])

In [ ]:
datetime.fromtimestamp(min(df_good_data['event_time'][:4000000])).strftime('%Y-%m-%d %H:%M:%S.%f')

In [ ]:
sorted_df = df_good_data.sort_values(by='event_time')

In [ ]:
sorted_df

In [ ]:
sorted_df['event_time'][11798122]

In [ ]:
datetime.fromtimestamp(1669165081.4727602).strftime('%Y-%m-%d %H:%M:%S.%f')

In [ ]:
for i in range(0,15):
    start = int(eval(f'{i}e6'))
    end = int(eval(f'{i+1}e6'))
    plt.figure(figsize=(12,12))
    alpha = 0.1
    plt.scatter(sorted_df['impact_x_array'][start:end], sorted_df['impact_y_array'][start:end], alpha = alpha, s = 1, marker = 'x')
    plt.xlabel("Impact parameter X [m]")
    plt.ylabel("Impact parameter Y [m]")
    plt.title(f"Reconstructed impact points of the muons for {str(start)[0]}e6-{str(end)[0]}e6 events on the LST mirror, transparency of dots = {alpha}")
    plt.show()
    #plt.savefig(f'/Users/vdk/muons2024/images/low_nsb_all_runs/test/ImpactPoints_{i}e6-{i+1}e6_Alpha{alpha}_SORTED.png', dpi=200, format='png', bbox_inches='tight')
    plt.close()

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(df_good_data['impact_x_array'][:400000], df_good_data['impact_y_array'][:400000], alpha = 0.05, s = 1, marker = 'x')
plt.xlabel("Impact parameter X [m]")
plt.ylabel("Impact parameter Y [m]")
plt.title("Reconstructed impact points of the muons on the LST mirror, transparency of dots = 1")
#plt.savefig('/Users/vdk/muons2024/images/ImpactPoints_19-23Y_Alpha1.png', dpi=200, format='png', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(df_good_data['impact_x_array'], df_good_data['impact_y_array'], alpha = 0.1, s = 1, marker = 'x')
plt.xlabel("Impact parameter X [m]")
plt.ylabel("Impact parameter Y [m]")
plt.title("Reconstructed impact points of the muons on the LST mirror, transparency of dots = 1")
#plt.savefig('/Users/vdk/muons2024/images/ImpactPoints_19-23Y_Alpha1.png', dpi=200, format='png', bbox_inches='tight')

In [ ]:
start = int(0)
end = int(12e6)
plt.figure(figsize=(12,12))
alpha = 0.01
plt.scatter(df_good_data['impact_x_array'][start:end], df_good_data['impact_y_array'][start:end], alpha = alpha, s = 1, marker = 'x')
plt.xlabel("Impact parameter X [m]")
plt.ylabel("Impact parameter Y [m]")
plt.title(f"Reconstructed impact points of the muons on the LST mirror, transparency of dots = {alpha}")
#plt.savefig('/Users/vdk/muons2024/images/ImpactPoints_10e6-12e6_Alpha005.png', dpi=200, format='png', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
#plt.figure(figsize=(12,12))
plt.scatter(df_good_data['impact_x_array'], df_good_data['impact_y_array'], alpha = 0.008, s = 1, marker = 'x')
plt.xlabel("Impact parameter X [m]")
plt.ylabel("Impact parameter Y [m]")
plt.title("Reconstructed impact points of the muons on the LST mirror, transparency of dots = 0.002")
circle1 = plt.Circle((0, 0), 5.55, color='r', fill=False, alpha = 1, lw = 0.25) # Circle with radius as artifact
ax.add_patch(circle1)
#plt.savefig('/Users/vdk/muons2024/images/ImpactPoints_19-23Y_Alpha0002.png', dpi=200, format='png', bbox_inches='tight')

In [ ]:
mean_opt_eff = np.mean(df_good_data['muon_efficiency'])
mean_ring_width = np.mean(df_good_data['ring_width'])
mean_ring_radius = np.mean(df_good_data['ring_radius'])
mean_ring_size = np.mean(df_good_data['ring_size'])
mean_impact_parameter = np.mean(df_good_data['impact_parameter'])
print(np.mean(df_good_data['muon_efficiency']))
print(np.mean(df_good_data['ring_width']))
print(mean_ring_radius)

In [ ]:
max(df_good_data['muon_efficiency'])

In [ ]:
x_value_str = 'ring_width'
y_value_str = 'muon_efficiency'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=500)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Plotting
plt.figure(figsize=(10, 6))

# Scatter plot of binned values
plt.scatter(binned_data['bin_mid'], binned_data[y_value_str], label='Binned Muon efficiency', s = 20)

# Calculate and plot the regression line over the original data
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
plt.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)

plt.xlabel(f'{x_value_str}')
plt.ylabel(f'{y_value_str}')
plt.axhline(y=mean_opt_eff, color='g', linestyle='--', label = f'Mean optical efficiency for this period = {round(mean_opt_eff, 3)}')
plt.axvline(x=mean_ring_width, color='orange', linestyle='--', label = f'Mean ring width for this period = {round(mean_ring_width,3)} deg')
plt.legend()
plt.ylim(0,1.42)
#plt.xlim(0,0.3)
plt.grid(alpha=0.5)
#plt.show()
#plt.savefig('/home/jovyan/XImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/home/jovyan/OptEffVSImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/Users/vdk/muons2024/images/WidthVsEfficiency_29-23Y_ylim2.png', dpi=200, format='png', bbox_inches='tight')

In [ ]:
x_value_str = 'ring_radius'
y_value_str = 'muon_efficiency'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=500)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Plotting
plt.figure(figsize=(10, 6))

# Scatter plot of binned values
plt.scatter(binned_data['bin_mid'], binned_data[y_value_str], label='Binned Muon efficiency', s = 20)

# Calculate and plot the regression line over the original data
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
plt.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)

plt.xlabel(f'{x_value_str}')
plt.ylabel(f'{y_value_str}')
plt.axhline(y=mean_opt_eff, color='g', linestyle='--', label = f'Mean optical efficiency for this period = {round(mean_opt_eff, 3)}')
plt.axvline(x=mean_ring_radius, color='orange', linestyle='--', label = f'Mean ring radius for this period = {round(mean_ring_radius,3)} deg')
plt.legend()
plt.ylim(0.15,0.2)
#plt.xlim(0,0.3)
plt.grid(alpha=0.5)
#plt.show()
#plt.savefig('/home/jovyan/XImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/home/jovyan/OptEffVSImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/Users/vdk/muons2024/images/RadiusVsEfficiency_19-23Y_ylim2.png', dpi=200, format='png', bbox_inches='tight')


In [ ]:
x_value_str = 'ring_size'
y_value_str = 'muon_efficiency'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=500)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Plotting
plt.figure(figsize=(10, 6))

# Scatter plot of binned values
plt.scatter(binned_data['bin_mid'], binned_data[y_value_str], label='Binned Muon efficiency', s=20)

# Calculate and plot the regression line over the original data
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
plt.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)

plt.xlabel(f'{x_value_str}')
plt.ylabel(f'{y_value_str}')
plt.axhline(y=mean_opt_eff, color='g', linestyle='--', label = f'Mean optical efficiency for this period = {round(mean_opt_eff, 3)}')
plt.axvline(x=mean_ring_size, color='orange', linestyle='--', label = f'Mean ring size for this period = {round(mean_ring_size,3)} pe')
plt.legend()
plt.ylim(0,1)
#plt.xlim(0,0.3)
plt.grid(alpha=0.5)
#plt.show()
#plt.savefig('/home/jovyan/XImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/home/jovyan/OptEffVSImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/Users/vdk/muons2024/images/SizeEfficiency_19-23Y_ylim2.png', dpi=200, format='png', bbox_inches='tight')

In [ ]:
x_value_str = 'impact_parameter'
y_value_str = 'muon_efficiency'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=500)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Plotting
plt.figure(figsize=(10, 6))

# Scatter plot of binned values
plt.scatter(binned_data['bin_mid'], binned_data[y_value_str], label='Binned Muon efficiency', s=20)

# Calculate and plot the regression line over the original data
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
plt.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)

plt.xlabel(f'{x_value_str}')
plt.ylabel(f'{y_value_str}')
plt.axhline(y=mean_opt_eff, color='g', linestyle='--', label = f'Mean optical efficiency for this period = {round(mean_opt_eff, 3)}')
#plt.axvline(x=mean_impact_parameter, color='orange', linestyle='--', label = f'Mean ring width for this period = {round(mean_impact_parameter,3)} pe')
plt.legend()
plt.ylim(0.15,0.3)
#plt.xlim(0,0.3)
plt.grid(alpha=0.5)
#plt.show()
#plt.savefig('/home/jovyan/XImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/home/jovyan/OptEffVSImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/Users/vdk/muons2024/images/ImpactEfficiency_19-23Y_ylim2.png', dpi=200, format='png', bbox_inches='tight')

In [ ]:
x_value_str = 'impact_x_array'
y_value_str = 'muon_efficiency'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=1000)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Plotting
plt.figure(figsize=(10, 6))

# Scatter plot of binned values
plt.scatter(binned_data['bin_mid'], binned_data[y_value_str], label='Binned Muon efficiency', s=20)

# Calculate and plot the regression line over the original data
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
plt.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)

plt.xlabel(f'{x_value_str}')
plt.ylabel(f'{y_value_str}')
plt.axhline(y=mean_opt_eff, color='g', linestyle='--', label = f'Mean optical efficiency for this period = {round(mean_opt_eff, 3)}')
#plt.axvline(x=mean_impact_parameter, color='orange', linestyle='--', label = f'Mean ring width for this period = {round(mean_impact_parameter,3)} pe')
plt.legend()
plt.ylim(0.15,0.3)
#plt.xlim(0,0.3)
plt.grid(alpha=0.5)
#plt.show()
#plt.savefig('/home/jovyan/XImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/home/jovyan/OptEffVSImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/Users/vdk/muons2024/images/ImpactEfficiency_19-23Y_ylim2.png', dpi=200, format='png', bbox_inches='tight')

In [ ]:
x_value_str = 'impact_y_array'
y_value_str = 'muon_efficiency'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=1000)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Plotting
plt.figure(figsize=(10, 6))

# Scatter plot of binned values
plt.scatter(binned_data['bin_mid'], binned_data[y_value_str], label='Binned Muon efficiency', s=20)

# Calculate and plot the regression line over the original data
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
plt.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)

plt.xlabel(f'{x_value_str}')
plt.ylabel(f'{y_value_str}')
plt.axhline(y=mean_opt_eff, color='g', linestyle='--', label = f'Mean optical efficiency for this period = {round(mean_opt_eff, 3)}')
#plt.axvline(x=mean_impact_parameter, color='orange', linestyle='--', label = f'Mean ring width for this period = {round(mean_impact_parameter,3)} pe')
plt.legend()
plt.ylim(0.15,0.3)
#plt.xlim(0,0.3)
plt.grid(alpha=0.5)
#plt.show()
#plt.savefig('/home/jovyan/XImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/home/jovyan/OptEffVSImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/Users/vdk/muons2024/images/ImpactEfficiency_19-23Y_ylim2.png', dpi=200, format='png', bbox_inches='tight')

In [ ]:
x_value_str = 'ring_radius'
y_value_str = 'ring_width'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=500)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Plotting
plt.figure(figsize=(10, 6))

# Scatter plot of binned values
plt.scatter(binned_data['bin_mid'], binned_data[y_value_str], label='Binned Ring Width', s=20)

# Calculate and plot the regression line over the original data
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
plt.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)

plt.xlabel(f'{x_value_str}')
plt.ylabel(f'{y_value_str}')
plt.axhline(y=mean_ring_width, color='g', linestyle='--', label = f'Mean width for this period = {round(mean_ring_width, 3)}')
plt.axvline(x=mean_ring_radius, color='orange', linestyle='--', label = f'Mean ring radius for this period = {round(mean_ring_radius,3)} pe')
plt.legend()
plt.ylim(0,0.2)
#plt.xlim(0,0.3)
plt.grid(alpha=0.5)
#plt.show()
#plt.savefig('/home/jovyan/XImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/home/jovyan/OptEffVSImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/Users/vdk/muons2024/images/RadiusVsWidth_19-23Y.png', dpi=200, format='png', bbox_inches='tight')

In [ ]:
x_value_str = 'ring_radius'
y_value_str = 'ring_size'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=500)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Plotting
plt.figure(figsize=(10, 6))

# Scatter plot of binned values
plt.scatter(binned_data['bin_mid'], binned_data[y_value_str], label='Binned Ring Size', s=20)

# Calculate and plot the regression line over the original data
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
plt.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)

plt.xlabel(f'{x_value_str}')
plt.ylabel(f'{y_value_str}')
plt.axhline(y=mean_ring_size, color='g', linestyle='--', label = f'Mean ring_size for this period = {round(mean_ring_size, 3)}')
plt.axvline(x=mean_ring_radius, color='orange', linestyle='--', label = f'Mean ring radius for this period = {round(mean_ring_radius,3)} pe')
plt.legend()
plt.ylim(0,5000)
#plt.xlim(0,0.3)
plt.grid(alpha=0.5)
#plt.show()
#plt.savefig('/home/jovyan/XImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/home/jovyan/OptEffVSImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/Users/vdk/muons2024/images/RadiusVsSize_19-23Y.png', dpi=200, format='png', bbox_inches='tight')

In [ ]:
x_value_str = 'event_time'
y_value_str = 'muon_efficiency'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=100)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Plotting
plt.figure(figsize=(10, 6))

# Scatter plot of binned values
plt.scatter(binned_data['bin_mid'], binned_data[y_value_str], label='Binned Muon Efficiency', s=20)

# Calculate and plot the regression line over the original data
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
plt.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)

plt.xlabel(f'{x_value_str}')
plt.ylabel(f'{y_value_str}')
#plt.axhline(y=mean_ring_size, color='g', linestyle='--', label = f'Mean ring_size for this period = {round(mean_ring_size, 3)}')
#plt.axvline(x=mean_ring_radius, color='orange', linestyle='--', label = f'Mean ring radius for this period = {round(mean_ring_radius,3)} pe')
plt.legend()
#plt.ylim(0,1.1)
#plt.xlim(0,0.3)
plt.grid(alpha=0.5)
#plt.show()
#plt.savefig('/home/jovyan/XImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/home/jovyan/OptEffVSImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/Users/vdk/muons2024/images/RadiusVsSize_19-23Y.png', dpi=200, format='png', bbox_inches='tight')

In [ ]:
mean_ring_size

In [ ]:
x_value_str = 'event_time'
y_value_str = 'ring_size'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=500)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Plotting
plt.figure(figsize=(10, 6))

# Scatter plot of binned values
plt.scatter(binned_data['bin_mid'], binned_data[y_value_str], label='Binned Muon Efficiency', s=20)

# Calculate and plot the regression line over the original data
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
plt.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)

plt.xlabel(f'{x_value_str}')
plt.ylabel(f'{y_value_str}')
#plt.axhline(y=mean_ring_size, color='g', linestyle='--', label = f'Mean ring_size for this period = {round(mean_ring_size, 3)}')
#plt.axvline(x=mean_ring_radius, color='orange', linestyle='--', label = f'Mean ring radius for this period = {round(mean_ring_radius,3)} pe')
plt.legend()
#plt.ylim(0,1.1)
#plt.xlim(0,0.3)
plt.grid(alpha=0.5)
#plt.show()
#plt.savefig('/home/jovyan/XImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/home/jovyan/OptEffVSImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/Users/vdk/muons2024/images/RadiusVsSize_19-23Y.png', dpi=200, format='png', bbox_inches='tight')

#### Plots with event count in each bin

In [ ]:

x_value_str = 'event_time'
y_value_str = 'ring_size'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=500)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Calculate number of elements in each bin
bin_counts = df_good_data.groupby(f'{x_value_str}_bin').size().reset_index(name='count')

# Merge the counts back with the mean calculations if needed
binned_data_with_counts = binned_data.merge(bin_counts, on=f'{x_value_str}_bin')


# Plotting
plt.figure(figsize=(12, 8))

# Scatter plot of binned values
plt.scatter(binned_data_with_counts['bin_mid'], binned_data_with_counts[y_value_str], label='Binned Ring Size', s=0.5 * binned_data_with_counts['count'], alpha = 0.8, marker='^')

# Calculate and plot the regression line over the original data
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
plt.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)

plt.xlabel(f'{x_value_str}')
plt.ylabel(f'{y_value_str}')
#plt.ylim(1400,2400)
plt.legend()
plt.grid(alpha=0.5)


In [ ]:
x_value_str = 'ring_width'
y_value_str = 'muon_efficiency'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=500)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Calculate number of elements in each bin
bin_counts = df_good_data.groupby(f'{x_value_str}_bin').size().reset_index(name='count')

# Merge the counts back with the mean calculations if needed
binned_data_with_counts = binned_data.merge(bin_counts, on=f'{x_value_str}_bin')


# Plotting
plt.figure(figsize=(10, 6))

# Scatter plot of binned values
plt.scatter(binned_data_with_counts['bin_mid'], binned_data_with_counts[y_value_str], label='Binned Muon Efficiency', s=0.5 * binned_data_with_counts['count'], alpha = 0.8)

# Calculate and plot the regression line over the original data
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
plt.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)

plt.xlabel(f'{x_value_str}')
plt.ylabel(f'{y_value_str}')
plt.legend()
plt.grid(alpha=0.5)


In [ ]:
x_value_str = 'impact_parameter'
y_value_str = 'muon_efficiency'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=100)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Calculate number of elements in each bin
bin_counts = df_good_data.groupby(f'{x_value_str}_bin').size().reset_index(name='count')

# Merge the counts back with the mean calculations if needed
binned_data_with_counts = binned_data.merge(bin_counts, on=f'{x_value_str}_bin')


# Plotting
plt.figure(figsize=(12, 8))

# Scatter plot of binned values
plt.scatter(binned_data_with_counts['bin_mid'], binned_data_with_counts[y_value_str], label='Binned Muon Efficiency', s=0.005 * binned_data_with_counts['count'], alpha = 0.8, marker = '^')

# Calculate and plot the regression line over the original data
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
plt.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)

plt.xlabel(f'{x_value_str}')
plt.ylabel(f'{y_value_str}')
#plt.ylim(0,0.6)
#plt.xlim(0,1)
plt.legend()
plt.grid(alpha=0.5)


In [ ]:
x_value_str = 'ring_radius'
y_value_str = 'ring_size'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=100)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Calculate number of elements in each bin
bin_counts = df_good_data.groupby(f'{x_value_str}_bin').size().reset_index(name='count')

# Merge the counts back with the mean calculations if needed
binned_data_with_counts = binned_data.merge(bin_counts, on=f'{x_value_str}_bin')


# Plotting
plt.figure(figsize=(10, 6))

# Scatter plot of binned values
plt.scatter(binned_data_with_counts['bin_mid'], binned_data_with_counts[y_value_str], label=f'Binned {y_value_str}', s=0.05 * binned_data_with_counts['count'], alpha = 0.8, marker = '^')

# Calculate and plot the regression line over the original data
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
plt.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)

plt.xlabel(f'{x_value_str}')
plt.ylabel(f'{y_value_str}')
# plt.ylim(0,0.6)
# plt.xlim(0,1)
plt.legend()
plt.grid(alpha=0.5)


In [ ]:
z,x,c = plt.hist(df_good_data['ring_width'], bins = 250)
plt.xlim(0,0.2)

In [ ]:
test_list = ['test1', 'test2', 'test3']

test_list.discard('test2')

In [ ]:
test_list = list(filter(lambda item: item!='test1',test_list))
test_list

In [ ]:
x_value_str = 'ring_radius'
y_value_str = 'impact_parameter'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=300)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Plotting
plt.figure(figsize=(10, 6))

# Scatter plot of binned values
plt.scatter(binned_data['bin_mid'], binned_data[y_value_str], label='Binned Muon efficiency', s = 20)

# Calculate and plot the regression line over the original data
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
plt.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)

plt.xlabel(f'{x_value_str}')
plt.ylabel(f'{y_value_str}')
#plt.axhline(y=mean_opt_eff, color='g', linestyle='--', label = f'Mean optical efficiency for this period = {round(mean_opt_eff, 3)}')
#plt.axvline(x=mean_ring_width, color='orange', linestyle='--', label = f'Mean ring width for this period = {round(mean_ring_width,3)} deg')
plt.legend()
#plt.ylim(0,1)
#plt.xlim(0,0.3)
plt.grid(alpha=0.5)
#plt.show()
#plt.savefig('/home/jovyan/XImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/home/jovyan/OptEffVSImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/Users/vdk/muons2024/images/WidthVsEfficiency_29-23Y_ylim2.png', dpi=200, format='png', bbox_inches='tight')

In [ ]:
x_value_str = 'ring_radius'
y_value_str = 'ring_completeness'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=1000)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Plotting
plt.figure(figsize=(10, 6))

# Scatter plot of binned values
plt.scatter(binned_data['bin_mid'], binned_data[y_value_str], label='Binned Muon efficiency', s = 20)

# Calculate and plot the regression line over the original data
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
plt.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)

plt.xlabel(f'{x_value_str}')
plt.ylabel(f'{y_value_str}')
#plt.axhline(y=mean_opt_eff, color='g', linestyle='--', label = f'Mean optical efficiency for this period = {round(mean_opt_eff, 3)}')
#plt.axvline(x=mean_ring_width, color='orange', linestyle='--', label = f'Mean ring width for this period = {round(mean_ring_width,3)} deg')
plt.legend()
#plt.ylim(0,1)
#plt.xlim(0,0.3)
plt.grid(alpha=0.5)
#plt.show()
#plt.savefig('/home/jovyan/XImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/home/jovyan/OptEffVSImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/Users/vdk/muons2024/images/WidthVsEfficiency_29-23Y_ylim2.png', dpi=200, format='png', bbox_inches='tight')

In [ ]:
usefull_func.hist_wo_outliers(df_good_data['impact_parameter'], outlier_value=10000, bin_number=50)

In [ ]:
usefull_func.hist_wo_outliers(df_good_data['impact_y_array'], outlier_value=10000, bin_number=50)

In [ ]:
usefull_func.hist_wo_outliers(df_good_data['impact_x_array'], outlier_value=10000, bin_number=50)